hi

In [1]:
import pandas as pd

# 1. Load the DataFrame
df = pd.read_excel(
    "data/WF 3 F1-R12 - Great Britain.xlsx",
    sheet_name="Worksheet",
    header=5
)

# 2. Define the columns to check and sort by
subset_cols = ['TV-Channel', 'Channel ID', 'Start', 'End', 'Region', 'Market', 'Duration' , 'Combined' , 'Broadcaster' , 'Program Description' , 'Program Title' ,'TVR% 3+' ,'Aud Metered (000s) 3+','Start (UTC)'  ]

# --- CRITICAL FIX: CONVERT TIME COLUMNS TO STRING (OBJECT) ---
# This resolves the 'datetime.datetime' vs 'datetime.time' comparison error.
for col in ['Start', 'End']:
    # The .apply(str) method is the most robust way to force all mixed contents to string.
    print(f"Standardizing column '{col}' to string type using .apply(str).")
    df[col] = df[col].apply(str)

# --- Secondary Fix (Addressing the Deprecation Warning and other 'category' issues) ---
# Ensure other non-time columns are not problematic 'category' dtypes
for col in subset_cols:
    # Using the modern isinstance check to address the DeprecationWarning
    if isinstance(df[col].dtype, pd.CategoricalDtype):
        print(f"Converting column '{col}' from 'category' to 'object' for sorting.")
        df[col] = df[col].astype('object')

# 3. Find all duplicate rows
duplicate_mask = df.duplicated(subset=subset_cols, keep=False)
duplicate_rows = df[duplicate_mask].copy()

# 4. Sort the DataFrame (This should now work reliably)
duplicate_rows = duplicate_rows.sort_values(by=subset_cols)

# Display the results
print(f"\nTotal duplicate rows found: {duplicate_rows.shape[0]}")
print("\nFirst 10 duplicate rows (based on TV-Channel, Channel ID, Start, End):")

# Columns for output display
display_cols = subset_cols + ['Date', 'Program Title']
print(duplicate_rows[display_cols].head(10))

Standardizing column 'Start' to string type using .apply(str).
Standardizing column 'End' to string type using .apply(str).

Total duplicate rows found: 147

First 10 duplicate rows (based on TV-Channel, Channel ID, Start, End):
                      TV-Channel  Channel ID     Start       End  Region  \
1674  Arenasport 1 (Pan Balkans)      5105.0  13:30:00  14:30:00  Europe   
1676  Arenasport 1 (Pan Balkans)      5105.0  13:30:00  14:30:00  Europe   
1219           Art Sport 6 (KOS)     39130.0  07:00:00  10:45:00  Europe   
1225           Art Sport 6 (KOS)     39130.0  07:00:00  10:45:00  Europe   
2973                      Canal+         NaN       nan       nan  Global   
2974                      Canal+         NaN       nan       nan  Global   
2975                      Canal+         NaN       nan       nan  Global   
2976                      Canal+         NaN       nan       nan  Global   
2977                      Canal+         NaN       nan       nan  Global   
2978       

In [4]:
import pandas as pd
import io

# 1. Dummy DataFrame for demonstration (Replace with your actual file load)
data = {
    'TV-Channel': ['A', 'A', 'B', 'B'],
    'Channel ID': [1, 1, 2, 2],
    'Start': ['10:00:00', '10:00:00', '11:00:00', '11:00:00'],
    'End': ['11:00:00', '11:00:00', '12:00:00', '12:00:00'],
    'Region': ['EU', 'EU', 'NA', 'NA'],
    'Market': ['Italy', 'Mexico', 'USA', 'USA'],
    'Duration': [60, 60, 60, 60],
    'Combined': ['C1', 'C1', 'C2', 'C2'],
    'Broadcaster': ['Sky', 'Sky', 'ESPN', 'ESPN'],
    'Program Description': ['PD1', 'PD1', 'PD2', 'PD2'],
    'Program Title': ['Race', 'Race', 'Race', 'Race'],
    'TVR% 3+': [2.5, 2.5, 3.0, 3.0],
    'Aud Metered (000s) 3+': [150, 150, 200, 200],
    'Start (UTC)': ['09:00:00', '09:00:00', '16:00:00', '16:00:00'],
    'Date': ['2025-01-01', '2025-01-01', '2025-01-01', '2025-01-01']
}
df = pd.DataFrame(data)

# 2. Define the columns to check
# *** KEY CHANGE: 'Market' is excluded to find duplicates across markets. ***
subset_cols_no_market = [
    'TV-Channel', 'Channel ID', 'Start', 'End', 'Region', 
    'Duration', 'Combined', 'Broadcaster', 'Program Description', 
    'Program Title', 'TVR% 3+', 'Aud Metered (000s) 3+', 'Start (UTC)'
]

# --- CRITICAL FIX: CONVERT TIME COLUMNS TO STRING (OBJECT) ---
for col in ['Start', 'End']:
    if col in df.columns:
        df[col] = df[col].apply(str)

# --- Secondary Fix: Convert 'category' dtypes to 'object' ---
for col in subset_cols_no_market:
    if col in df.columns and isinstance(df[col].dtype, pd.CategoricalDtype):
        df[col] = df[col].astype('object')

# 3. Find all duplicate rows 
duplicate_mask = df.duplicated(subset=subset_cols_no_market, keep=False)
duplicate_rows = df[duplicate_mask].copy()

# 4. Sort the DataFrame 
duplicate_rows = duplicate_rows.sort_values(by=['Market'] + subset_cols_no_market)

# 5. Display the results with proper summary
total_duplicates = duplicate_rows.shape[0]
total_unique_duplicate_sets = total_duplicates // 2 if total_duplicates > 0 else 0

# Columns for output display
display_cols = ['Market'] + subset_cols_no_market + ['Date', 'Program Title']

print("\n" + "="*80)
print("                           CROSS-MARKET DUPLICATE CHECK RESULTS                         ")
print("="*80)
print(f"**LOGIC:** Duplicate search was performed on ALL columns EXCEPT 'Market'.")
print(f"**Total duplicate rows found (rows with identical content):** {total_duplicates}")
print(f"**Estimated unique sets of duplicates:** {total_unique_duplicate_sets}")
print("="*80)
print("\nFirst 10 duplicate rows (sorted by Market then content to easily spot cross-market pairs):")

# Print the table output
print(duplicate_rows[display_cols].head(10).to_markdown(index=False))


                           CROSS-MARKET DUPLICATE CHECK RESULTS                         
**LOGIC:** Duplicate search was performed on ALL columns EXCEPT 'Market'.
**Total duplicate rows found (rows with identical content):** 4
**Estimated unique sets of duplicates:** 2

First 10 duplicate rows (sorted by Market then content to easily spot cross-market pairs):
| Market   | TV-Channel   |   Channel ID | Start    | End      | Region   |   Duration | Combined   | Broadcaster   | Program Description   | Program Title   |   TVR% 3+ |   Aud Metered (000s) 3+ | Start (UTC)   | Date       | Program Title   |
|:---------|:-------------|-------------:|:---------|:---------|:---------|-----------:|:-----------|:--------------|:----------------------|:----------------|----------:|------------------------:|:--------------|:-----------|:----------------|
| Italy    | A            |            1 | 10:00:00 | 11:00:00 | EU       |         60 | C1         | Sky           | PD1                   | Race 

In [6]:
import pandas as pd
import sys

# 1. File Path and Loading (Using your specified file path)
FILE_PATH = "data/WF 3 F1-R12 - Great Britain.xlsx"
SHEET_NAME = "Worksheet"
HEADER_ROW_INDEX = 5 

try:
    df = pd.read_excel(
        FILE_PATH,
        sheet_name=SHEET_NAME,
        header=HEADER_ROW_INDEX
    )
except FileNotFoundError:
    print(f"\n❌ ERROR: File not found at the specified path: {FILE_PATH}")
    print("Please ensure the file exists and the path is correct before running locally.")
    sys.exit(1) 

# 2. Define the columns to check
# The 'Market' column is intentionally EXCLUDED to find duplicates across markets.
subset_cols_no_market = [
    'TV-Channel', 'Channel ID', 'Start', 'End', 'Region', 
    'Duration', 'Combined', 'Broadcaster', 'Program Description', 
    'Program Title', 'TVR% 3+', 'Aud Metered (000s) 3+', 'Start (UTC)'
]

# --- Data Type Standardization (Critical Fixes) ---
for col in ['Start', 'End']:
    if col in df.columns:
        df[col] = df[col].apply(str)

for col in subset_cols_no_market:
    if col in df.columns and isinstance(df[col].dtype, pd.CategoricalDtype):
        df[col] = df[col].astype('object')

# 3. Find all duplicate rows
duplicate_mask = df.duplicated(subset=subset_cols_no_market, keep=False)
duplicate_rows = df[duplicate_mask].copy()

# 4. Preparation and Exit if no duplicates are found
if duplicate_rows.empty:
    print("\n================================================================================")
    print("                           CROSS-MARKET DUPLICATE CHECK                         ")
    print("================================================================================")
    print("STATUS: SUCCESS. No cross-market content duplicates were found.")
    sys.exit(0)

# Create a unique identifier for each duplicate set (grouping by content)
duplicate_rows['Content_ID'] = duplicate_rows.groupby(subset_cols_no_market).ngroup()

# 5. Generate Detailed Summary Statistics (THE MARKET GROUPING)
duplicate_summary = duplicate_rows.groupby('Content_ID')['Market'].agg([
    ('Duplicate Count', 'size'), 
    ('Markets Involved', lambda x: ', '.join(sorted(x.unique())))
]).reset_index()

# Merge key content columns back to the summary for context
key_content_cols = ['Program Title', 'Broadcaster', 'Date']
content_map = duplicate_rows.drop_duplicates(subset=subset_cols_no_market).set_index('Content_ID')[key_content_cols]
duplicate_summary = duplicate_summary.merge(content_map, left_on='Content_ID', right_index=True, how='left')
duplicate_summary = duplicate_summary.drop(columns='Content_ID')

# Reorder columns and sort by count
duplicate_summary = duplicate_summary[['Duplicate Count', 'Markets Involved'] + key_content_cols]
duplicate_summary = duplicate_summary.sort_values(by='Duplicate Count', ascending=False)

# 6. Display the results
total_duplicates = duplicate_rows.shape[0]
total_unique_duplicate_sets = duplicate_summary.shape[0]

display_cols = ['Market'] + subset_cols_no_market + ['Date', 'Program Title']

print("\n" + "="*80)
print("                           CROSS-MARKET DUPLICATE CHECK RESULTS                         ")
print("================================================================================")
print(f"**FILE CHECKED:** {FILE_PATH} (Sheet: {SHEET_NAME})")
print(f"**Total duplicate rows found (rows with identical content):** {total_duplicates}")
print(f"**Total unique duplicate sets (distinct broadcast content):** {total_unique_duplicate_sets}")
print("================================================================================")

print("\n--- DETAILED DUPLICATION SUMMARY (MARKET GROUPS) ---")
print("This table shows the **Market Groups** formed by content duplication:")
print(duplicate_summary.to_markdown(index=False))

print("\n" + "="*80)
print("INDIVIDUAL DUPLICATED ROWS (Sorted by Market for Manual Review):")
# Sort the duplicate rows for easy manual review
duplicate_rows = duplicate_rows.sort_values(by=['Market'] + subset_cols_no_market)
print(duplicate_rows[display_cols].head(20).to_markdown(index=False))


                           CROSS-MARKET DUPLICATE CHECK RESULTS                         
**FILE CHECKED:** data/WF 3 F1-R12 - Great Britain.xlsx (Sheet: Worksheet)
**Total duplicate rows found (rows with identical content):** 640
**Total unique duplicate sets (distinct broadcast content):** 0

--- DETAILED DUPLICATION SUMMARY (MARKET GROUPS) ---
This table shows the **Market Groups** formed by content duplication:
| Duplicate Count   | Markets Involved   | Program Title   | Broadcaster   | Date   |
|-------------------|--------------------|-----------------|---------------|--------|

INDIVIDUAL DUPLICATED ROWS (Sorted by Market for Manual Review):
| Market   | TV-Channel      |   Channel ID | Start    | End      | Region   | Duration   | Combined                                                | Broadcaster   |   Program Description | Program Title                                           |   TVR% 3+ |   Aud Metered (000s) 3+ | Start (UTC)   | Date                | Program Title      

In [8]:
import pandas as pd
import sys

# --- CONFIGURATION ---
FILE_PATH = "data/WF 3 F1-R12 - Great Britain.xlsx"
SHEET_NAME = "Worksheet"
HEADER_ROW_INDEX = 5 
MARKET_A = 'Italy'
MARKET_B = 'Mexico'

# Define the columns used for the TIGHT comparison (channels, duration, broadcaster)
# *** CHANGE: 'Program Title' is explicitly removed from the CHECK_COLS ***
CHECK_COLS = ['TV-Channel', 'Duration', 'Broadcaster'] 
SUMMARY_COLS = ['TV-Channel', 'Duration', 'Broadcaster'] # Columns to display in the summary table

try:
    df = pd.read_excel(
        FILE_PATH,
        sheet_name=SHEET_NAME,
        header=HEADER_ROW_INDEX
    )
except FileNotFoundError:
    print(f"\n❌ ERROR: File not found at the specified path: {FILE_PATH}")
    print("Please ensure the file exists and the path is correct before running locally.")
    sys.exit(1)

# 1. Prepare and Filter Data
# Standardize key comparison columns to prevent type errors
for col in CHECK_COLS:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.upper()

# Filter data for only the two target markets
df_filtered = df[df['Market'].isin([MARKET_A, MARKET_B])].copy()

if df_filtered.empty:
    print(f"\nNo data found for markets {MARKET_A} or {MARKET_B}. Check spelling or data source.")
    sys.exit(0)

# 2. Identify Matching Lines (Content Duplicates)
# Find content lines (based on CHECK_COLS) that exist in BOTH markets
duplicate_mask = df_filtered.duplicated(subset=CHECK_COLS, keep=False)
matching_rows = df_filtered[duplicate_mask].copy()

# A set of keys that exist in both Italy and Mexico (Cross-Market Matches)
matching_keys = matching_rows.groupby(CHECK_COLS).filter(
    lambda x: set(x['Market'].unique()) == {MARKET_A, MARKET_B}
)

# 3. Calculate Summary Statistics
# Note: total_matched_lines is based on unique content keys, divided by 2 because each match is counted twice
total_matched_lines = matching_keys.drop_duplicates(subset=CHECK_COLS).shape[0]
total_rows_A = df_filtered[df_filtered['Market'] == MARKET_A].shape[0]
total_rows_B = df_filtered[df_filtered['Market'] == MARKET_B].shape[0]

# Rows that are unique to each market (not matched content)
unmatched_rows_A = total_rows_A - matching_keys[matching_keys['Market'] == MARKET_A].drop_duplicates(subset=CHECK_COLS).shape[0]
unmatched_rows_B = total_rows_B - matching_keys[matching_keys['Market'] == MARKET_B].drop_duplicates(subset=CHECK_COLS).shape[0]


# 4. Display Results
print("\n" + "="*80)
print(f"                       MARKET COMPARISON: {MARKET_A.upper()} vs {MARKET_B.upper()}                   ")
print("================================================================================")
print(f"**COMPARISON CRITERIA (MATCHING):** '{', '.join(CHECK_COLS)}'.")
print(f"Total rows in {MARKET_A}: {total_rows_A}")
print(f"Total rows in {MARKET_B}: {total_rows_B}")
print("================================================================================")

print("\n--- MATCHING CONTENT SUMMARY ---")
print(f"✅ **Total unique content lines matching in BOTH markets:** {total_matched_lines}")

# List the key details for the matched content
matched_content_df = matching_keys.drop_duplicates(subset=CHECK_COLS).head(10).copy()
matched_content_df['Status'] = 'MATCHED'
print("\nFirst 10 Matched Content Lines:")
print(matched_content_df[['Status'] + SUMMARY_COLS].to_markdown(index=False))

print("\n--- UNMATCHED CONTENT SUMMARY ---")
print(f"❌ **Unique content lines found ONLY in {MARKET_A}:** {unmatched_rows_A}")
print(f"❌ **Unique content lines found ONLY in {MARKET_B}:** {unmatched_rows_B}")

# Display the unmatched rows for Italy (up to 5)
unmatched_A_df = df_filtered.groupby(CHECK_COLS).filter(lambda x: MARKET_A in x['Market'].values and MARKET_B not in x['Market'].values)
print(f"\nFirst 5 unique lines found ONLY in {MARKET_A}:")
print(unmatched_A_df.drop_duplicates(subset=CHECK_COLS).head(5)[SUMMARY_COLS + ['Program Title']].to_markdown(index=False))

# Display the unmatched rows for Mexico (up to 5)
unmatched_B_df = df_filtered.groupby(CHECK_COLS).filter(lambda x: MARKET_B in x['Market'].values and MARKET_A not in x['Market'].values)
print(f"\nFirst 5 unique lines found ONLY in {MARKET_B}:")
print(unmatched_B_df.drop_duplicates(subset=CHECK_COLS).head(5)[SUMMARY_COLS + ['Program Title']].to_markdown(index=False))


                       MARKET COMPARISON: ITALY vs MEXICO                   
**COMPARISON CRITERIA (MATCHING):** 'TV-Channel, Duration, Broadcaster'.
Total rows in Italy: 184
Total rows in Mexico: 39

--- MATCHING CONTENT SUMMARY ---
✅ **Total unique content lines matching in BOTH markets:** 0

First 10 Matched Content Lines:
| Status   | TV-Channel   | Duration   | Broadcaster   |
|----------|--------------|------------|---------------|

--- UNMATCHED CONTENT SUMMARY ---
❌ **Unique content lines found ONLY in Italy:** 184
❌ **Unique content lines found ONLY in Mexico:** 39

First 5 unique lines found ONLY in Italy:
| TV-Channel         | Duration   | Broadcaster   | Program Title                      |
|:-------------------|:-----------|:--------------|:-----------------------------------|
| SKY SPORT 24 IT    | 00:15:53   | SKY           | F1 Paddock Live Pit Walk 2025 2025 |
| SKY SPORT 24 IT    | 00:12:48   | SKY           | Reparto Corse F1 2025 2025         |
| SKY SPORT F1 (ITA